In [ ]:
from __future__ import annotations

import datetime
import math
import numpy as np
import time
from sklearn.linear_model import LinearRegression
from common import load_csv, Route, Checkpoint, Coordinate, solar_power_out

In [ ]:
#constants 
dayMs = 1000 * 60 * 60 * 24
J1970 = 2440588
J2000 = 2451545
rad = math.pi / 180.0
e = rad * 23.4397
CELL_AREA = 0.0153 #m^2

In [ ]:
#utility functions
def altitude(H: float, phi: float, dec: float):
    return math.asin(math.sin(phi) * math.sin(dec) + math.cos(phi) * math.cos(dec) * math.cos(H))

def azimuth(H: float, phi: float, dec: float):  
    return math.atan2(math.sin(H), math.cos(H) * math.sin(phi) - math.tan(dec) * math.cos(phi))

def siderealTime(d: float, lw: float):
     return rad * (280.16 + 360.9856235 * d) - lw

def rightAscension(l: float, b: float): 
    return math.atan2(math.sin(l) * math.cos(e) - math.tan(b) * math.sin(e), math.cos(l))

def declination(l: float, b: float):    
    return math.asin(math.sin(b) * math.cos(e) + math.cos(b) * math.sin(e) * math.sin(l))

def toJulian(date: datetime.datetime):
    return (time.mktime(date.timetuple()) * 1000) / dayMs - 0.5 + J1970

def toDays(date: datetime.datetime):   
    return toJulian(date) - J2000

def solarMeanAnomaly(d: float):
    return rad * (357.5291 + 0.98560028 * d)

def eclipticLongitude(M: float):
    C = rad * (1.9148 * math.sin(M) + 0.02 * math.sin(2 * M) + 0.0003 * math.sin(3 * M)) # equation of center
    P = rad * 102.9372 # perihelion of the Earth
    return M + C + P + math.pi

def sunCoords(d: float):
    M = solarMeanAnomaly(d)
    L = eclipticLongitude(M)
    return dict(
        dec= declination(L, 0),
        ra= rightAscension(L, 0)
    )

def getSunPosition(date: datetime.datetime, lat: float, lon: float):
    """Returns positional attributes of the sun for the given time and location."""
    lw  = rad * -lon
    phi = rad * lat
    d   = toDays(date)

    c  = sunCoords(d)
    H  = siderealTime(d, lw) - c["ra"]

    return dict(
        azimuth=azimuth(H, phi, c["dec"]),
        altitude=altitude(H, phi, c["dec"])
    )


In [ ]:
#absolute cell tilts 
tilts = {
    "hood_front": {
        "left_stairs": [33.38, 27.64, 27.64, 27.63, 27.63, 27.63, 27.44, 27.44, 27.44],
        "left_center_3x4": [33.38, 33.38, 33.38, 27.64, 27.64, 27.64, 27.63, 27.63, 27.63, 27.44, 27.44, 27.44],
        "right_center_3x4": [33.38, 33.38, 33.38, 27.64, 27.64, 27.64, 27.63, 27.63, 27.63, 27.44, 27.44, 27.44],
        "right_stairs": [33.38, 27.64, 27.64, 27.63, 27.63, 27.63, 27.44, 27.44, 27.44]
    },
    "top_front": {
        "leftmost_top_3x2": [28.52, 28.52, 28.52, 22.16, 22.16, 22.16],
        "leftmost_center_3x2": [22.16, 22.16, 22.16, 22.16, 22.16, 22.16],
        "leftmost_bottom_3x2": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14],
        "leftcenter_top_3x2": [28.52, 28.52, 28.52, 22.16, 22.16, 22.16],
        "leftcenter_center_3x2": [22.16, 22.16, 22.16, 22.16, 22.16, 22.16],
        "leftcenter_bottom_3x2": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14],
        "rightcenter_top_3x2": [28.52, 28.52, 28.52, 22.16, 22.16, 22.16],
        "rightcenter_center_3x2": [22.16, 22.16, 22.16, 22.16, 22.16, 22.16],
        "rightcenter_bottom_3x2": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14],
        "rightmost_top_4x3": [28.52, 28.52, 28.52, 28.52, 22.16, 22.16, 22.16, 22.16, 22.16, 22.16, 22.16, 22.16],
        "rightmost_bottom_4x3": [22.16, 22.16, 22.16, 22.16, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14]
    },
    "top_back": {
        "leftmost_3x4": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 1.11, 1.11, 1.11, -3.64, -3.64, -3.64],
        "leftcenter_3x4": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 1.11, 1.11, 1.11, -3.64, -3.64, -3.64],
        "rightcenter_3x4": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 1.11, 1.11, 1.11, -3.64, -3.64, -3.64],
        "rightmost_4x4": [13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 13.14, 1.11, 1.11, 1.11, 1.11, -3.64, -3.64, -3.64, -3.64]
    },
    "back": {
        "leftmost_top_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
        "leftcenter_top_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
        "rightcenter_top_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
        "rightmost_top_4x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
        "leftmost_bottom_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
        "leftcenter_bottom_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64], 
        "rightcenter_bottom_3x4": [-3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64, -3.64],
        "rightmost_bottom-4x4": [-4, -4, -4, -4, -5, -5, -5, -5, -6, -6, -6, -6]
    }
}

In [ ]:
def cell_solar_irradiance(lat, lon, time, elevation, module_tilt, car_azimuth_angle):
    
    sunPosition = getSunPosition(time, lat, lon)
#     print("sunPosition: ", sunPosition)
    sun_elevation_angle = math.degrees(sunPosition['altitude'])
    if sun_elevation_angle < 0:
        sun_elevation_angle = 360 + sun_elevation_angle
#     print("sun_elevation_angle (deg): ", sun_elevation_angle)
    sun_azimuth_angle = math.degrees(sunPosition['azimuth'])
    if sun_azimuth_angle < 0:
        sun_azimuth_angle = 360 + sun_azimuth_angle
#     print("sun_azimuth_angle (deg): ", sun_azimuth_angle)
    
    #https://www.pveducation.org/pvcdrom/properties-of-sunlight/air-mass#AMequation
    #s_incident measured in kW/m^2 should be retrieved empirically
    air_mass = 1/(math.cos(math.radians(90-sun_elevation_angle)) + 0.50572*(96.07995-(90-sun_elevation_angle))**-1.6364) 
#     print("air_mass: ", air_mass)
#     print("elevation (km): ", elevation)
    
    s_incident_diffuse = 1.1*1.353*((1-0.14*elevation)*0.7**air_mass**0.678 + 0.14*elevation)
    print("s_incident_diffuse (kW/m^2): ", s_incident_diffuse)
    
    s_cell_diffuse = s_incident_diffuse*(math.cos(math.radians(sun_elevation_angle))*math.sin(math.radians(module_tilt))*math.cos(math.radians(car_azimuth_angle - sun_azimuth_angle)) 
                     + math.sin(math.radians(sun_elevation_angle))*math.cos(math.radians(module_tilt)))
#     print("s_cell_diffuse (kW/m^2): ", s_cell_diffuse)
#     print("s_cell (kW): ", s_cell_diffuse*cell_area)
    
    return s_cell_diffuse*cell_area

def section_solar_irradiance(lat, lon, time, elevation, module_tilts, car_azimuth_angle):
    total_sum = 0
    #store solar irradiance value of similar angles in a dictionary and use it to approximate?
    #definitely keep reference of same values to limit api calls
    for module in module_tilts.keys():
        module_sum = 0
        for cell_angle in module_tilts[module]:
            cell_irr = cell_solar_irradiance(lat, lon, time, elevation, cell_angle, car_azimuth_angle)
            print("cell_irr: ", cell_irr)
            module_sum += cell_irr
        print("module sum: ", module_sum)
        total_sum += module_sum
    return total_sum

def total_irradiance(lat, lon, time, elevation, module_tilts, car_azimuth_angle):
    pass
    #sum of all the modules on the car

def energy_captured_along_route(time_initial: datetime.time, velocity: float, route: list[Coordinate]):
    pass
    #sum of all 

In [ ]:
class CellSolarData:
    def __init__(self, coord: Checkpoint, time: datetime.time, tilt: float):
        self.heading_azimuth_angle = coord.azimuth
        self.heading_azimuth_angle %= 360
        self.tilt = tilt
        self.tilt %= 360
        self.lat = coord.lat
        self.lon = coord.lon
        self.elevation = coord.elevation / 1000 #convert to km
        self.time = time
        sun_position = getSunPosition(time, coord.lat, coord.lon)
        self.sun_elevation_angle = math.degrees(sun_position['altitude'])
        self.sun_elevation_angle %= 360
        self.sun_azimuth_angle = math.degrees(sun_position['azimuth'])
        self.sun_azimuth_angle %= 360
        self.air_mass = abs(1/(math.cos(math.radians(90-self.sun_elevation_angle)) + 0.50572*(96.07995-(90-self.sun_elevation_angle))**-1.6364))
        self.incident_diffuse = 1.1*1.353*((1-0.14*(self.elevation / 1000))*0.7**self.air_mass**0.678 + 0.14*(self.elevation / 1000)) 
        self.cell_diffuse = self.incident_diffuse*(math.cos(math.radians(self.sun_elevation_angle))*math.sin(math.radians(self.tilt))*math.cos(math.radians(self.heading_azimuth_angle - self.sun_azimuth_angle)) + math.sin(math.radians(self.sun_elevation_angle))*math.cos(math.radians(self.tilt)))
        self.cell_irradiance = self.cell_diffuse * 1000 #convert to watts/m^2
        self.cloud_cover = coord.cloud_cover / 100
        self.cell_power_out = solar_power_out(self.cell_irradiance, self.cloud_cover) * CELL_AREA #watts
    

In [ ]:
def section_solar_power_out(coord: Checkpoint, time: datetime.time, section_tilts: dict):
    section_irradiance_sum = 0
    tilt_irradiances: dict[float, CellSolarData] = {}
    for array in section_tilts.keys():
        array_sum = 0
        for cell_angle in section_tilts[array]:
            if(cell_angle in tilt_irradiances):
                array_sum += tilt_irradiances[cell_angle].cell_power_out
            else:
                cell = CellSolarData(coord, time, cell_angle)
                tilt_irradiances[cell_angle] = cell
                array_sum += cell.cell_power_out
        section_irradiance_sum += array_sum
    #watts
    return section_irradiance_sum

def total_solar_power_out(coord: Checkpoint, time: datetime.time, tilts: dict):
    # iterating over all of the sections in the car
    car_power_sum: float = 0
    for section in tilts.keys():
        car_power_sum += section_solar_power_out(coord, time, tilts[section])
    #watts
    return car_power_sum

    #sum of all the modules on the car

def energy_captured_along_route(time_initial: datetime.time, velocity: float, route: Route):
    current_time = time_initial
    total_energy = 0
    velocity_ms = velocity / 3.6 #convert to m/2
    power_list = []
    for i in range(len(route.checkpoints)-2):
        segment_distance = route.checkpoints[i+1].distance if i==0 else route.checkpoints[i+1].distance - route.checkpoints[i].distance

        segment_time = segment_distance/velocity_ms

        total_power_in_current = total_solar_power_out(route.checkpoints[i], current_time, tilts=tilts)
        current_time += datetime.timedelta(seconds=segment_time)
        total_power_in_next = total_solar_power_out(route.checkpoints[i+1], current_time, tilts=tilts)
        total_power_in_avg = (total_power_in_current + total_power_in_next) / 2
        power_list.append(total_power_in_avg)
        total_energy += total_power_in_avg*segment_time #watt seconds
   
    print(time_initial, current_time)
    return (total_energy/3600, power_list) #in watt hours


In [ ]:
#equation to calculate distance between coordinates 
def heversine(lon1: float, lat1: float, lon2: float, lat2: float):
    R = 6371.0  #Radius of the Earth in km
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    
    return distance

In [ ]:
air_density = 1.204 #kg/m^3
coef_drag = 0.19
coef_rr = 0.0023
car_mass = 575 #kg
accel_g = 9.81 #m/s^2
wheel_radius = 0.2 #m
wind_speed = 0 #km/h
cross_section = 2.21 #m^2

def calculate_work(grad: float, speed_initial: float, speed_end: float, time: float):
    _speed_0 = 0.277777778*speed_initial #m/s
    _speed_1 = 0.277777778*speed_end #m/s
    accel = (_speed_1 - _speed_0) / time
    force_rr = coef_rr*car_mass*accel_g*math.cos(math.radians(grad))
    force_drag = 0.5*air_density*coef_drag*((_speed_0 + _speed_1)/2 + wind_speed)**2
    force_grad = car_mass*accel_g*math.sin(math.radians(grad))
    force_accel = car_mass*accel
    accel_distance = time*_speed_0 + (accel*time**2)/2
    force_tractive = force_rr + force_drag + force_grad + force_accel
    work = accel_distance * force_tractive

    return work #J

In [ ]:
def predict_average_electrical_power(avg_tractive_power: float):
    efficiencyDataX = np.array([0, 16.688, 152.532, 246.05, 335.306, 423.94, 513.536, 677.019, 
                            840.528, 996.03, 1147.608, 1278.624, 1405.316, 1537.61, 1783.478, 1974.78]).reshape((-1,1))
    efficiencyDataY = np.array([0, 28, 171, 266, 359, 451, 544, 721, 898, 1071, 1242, 1402, 1558, 1718, 2069, 2385])
    effModel = LinearRegression()
    effModel.fit(efficiencyDataX, efficiencyDataY)

    # r_sq = effModel.score(efficiencyDataX, efficiencyDataY)
    # _b = effModel.intercept_
    # _m = effModel.coef_

    avg_electrical_power = effModel.predict(np.array([avg_tractive_power]).reshape((-1,1)))

    return avg_electrical_power[0] #Watts

In [ ]:
def energy_used_between_points(c1: Coordinate, c2: Coordinate, speed_initial: float, speed_end: float, time: float):
    delta_elev = c2.elevation - c1.elevation
    delta_d = heversine(c1.lon, c1.lat, c2.lon, c2.lat)
    theta = math.degrees(math.atan(delta_elev/delta_d))

    work = calculate_work(theta, speed_initial, speed_end, time)
    avg_tractive_power = work / time
    avg_electrical_power = predict_average_electrical_power(avg_tractive_power)
    electrical_energy = avg_electrical_power * time
    
    return electrical_energy

In [ ]:
def energy_used_along_route(route: list[Coordinate]):
    total = 0
    
    for i in range(1, route):
        total += energy_used_between_points(route[i-1], route[i], 20, 20, 10)

    return total

In [ ]:

route = load_csv("A. Independence to Topeka")
_,powers = energy_captured_along_route(datetime.datetime.now(), 56.33, route)

In [ ]:
import folium
import branca.colormap
import pandas as pd
# from itertools import zip_longest

def route_to_list(route: Route):
    coords: list[tuple[float, float]] = [] 
    for checkpoint in route.checkpoints:
        coords.append((checkpoint.lat, checkpoint.lon))
    return coords

def create_map(route: Route, power_list: list):
    coords = route_to_list(route)
    m = folium.Map()
    colormap = branca.colormap.linear.YlOrRd_09.scale(min(power_list), max(power_list)).to_step(6)
    # tooltip = [str(i) for i in power_list]
    folium.ColorLine(positions=coords, colormap=colormap, weight=5, colors=power_list).add_to(m)
    # for i,p in zip_longest(power_list, coords, fillvalue=np.mean(power_list)):
    #     folium.Marker(p, tooltip = i).add_to(m)
    df = pd.DataFrame(coords).rename(columns={0: 'lat', 1: 'lon'})
    print(df)
    sw = df[['lat', 'lon']].min().values.tolist()
    ne = df[['lat', 'lon']].max().values.tolist()
    m.fit_bounds([sw, ne])
    m.add_child(colormap)
    return m
m = create_map(route, powers)
m

In [ ]:
VOLTAGE = 96 #V
def estimate_soc(q_initial: float, time_initial: datetime.time, velocity: float, route: Route):
    q_final = q_initial
    return q_final